<a href="https://colab.research.google.com/github/stevegaughan/lakeCoSTR_colab/blob/main/RAG_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-xterm
!pip install langchain langchain-core langchain-community gradio requests beautifulsoup4 transformers sentence-transformers chromadb
!pip install dartmouth-langchain
%load_ext colabxterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 

In [ ]:
from dartmouth_langchain.llms import DartmouthChatModel
from langchain.prompts import PromptTemplate


llm = DartmouthChatModel(dartmouth_api_key= "wjNaLe5rpUDtrwekbq70rDvauad8juXrH9wiMRDzgU2EIRAUcmpkffgu7uSqdMYa8CnLSFuO1VdO8MrEz5a1Fb40o5Z1ECDGcH6JbhwpYPkuvWNJkpBvNQNpTrDd0tgTSCrywg6zV3eZLOcQXlRU5G86ZaTtdsWZWqLjTOno7z4itFz38hHsCF9lbCbbInLfhUnteQlpJczOToiWrWvVc0MudKc83q3NzBMfBZGEOnWP5TBm5UBQNeMBm5l8Cqq11PRL21bxMwvyzqS6x4IE2L9gXbjzIwNToycQ63fp9w36lVw2dczdigyIrtBlOSun9X31LdQdQm3NQrtRKgKvxfuZ",
                         model_name="llama-3-8b-instruct"
                         )
prompt = PromptTemplate.from_template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a friendly ai who answers questions based on context<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: {question}
Context: {context}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""")

question = "what time is it?"
context = "Current local time is 1:50pm"
chain = prompt | llm

chain.invoke({"question": question, "context": context})


/usr/local/lib/python3.10/dist-packages/dartmouth_langchain/llms.py:79: UserWarning: DartmouthChatModel is deprecated and will be removed in a future update. Use `DartmouthLLM` (as a drop-in replacement) or `ChatDartmouth` instead!
  warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


'\n\nAccording to the current local time, it is 1:50 pm!'

In [ ]:
import requests
import gradio as gr
from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

from langchain_community.vectorstores import Chroma

from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer


# Custom embedding class for SentenceTransformer to be compatible with Chroma
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents):
        # Ensure documents are a list of strings
        if isinstance(documents[0], str):
            texts = documents
        else:
            texts = [doc.page_content for doc in documents]
        # Convert numpy arrays to lists
        embeddings = self.model.encode(texts)
        return embeddings.tolist()

    def embed_query(self, text):
        # Convert numpy array to list
        embedding = self.model.encode([text])
        return embedding[0].tolist()


# Load the data from the web URLdatabases
url = 'https://plato.stanford.edu/entries/ethics-ai/'  # Replace with your actual URL
print("Loading data from the web URL...")
loader = WebBaseLoader(url)
docs = loader.load()
print("Data loaded.")

# Split the loaded documents into chunks
print("Splitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print("Documents split into chunks.")

# Extract text content from Document objects for embedding
split_texts = [doc.page_content for doc in splits]

# Create embeddings using sentence-transformers
print("Creating embeddings...")
embedding_model = SentenceTransformerEmbeddings('all-MiniLM-L6-v2')

# Create vector store
print("Creating vector store...")
vectorstore = Chroma.from_texts(texts=split_texts, embedding=embedding_model)
print("Vector store created.")

# Define the RAG setup
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    )
rag_chain.invoke("Can you explain the key principles of AI ethics?")

Loading data from the web URL...
Data loaded.
Splitting documents into chunks...
Documents split into chunks.
Creating embeddings...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating vector store...
Vector store created.


'\n\nBased on the provided context, the key principles of AI ethics can be summarized as follows:\n\n1. AI ethics is not just about avoiding harm or preventing negative consequences, but rather about navigating complex moral dilemmas where the right answer is not readily known.\n2. AI systems can be viewed as both objects (tools made and used by humans) and subjects (having their own agency and decision-making capabilities).\n3. Ethical issues arise in both cases, including:\n\t* Object-level issues: privacy, manipulation, opacity, bias, human-robot interaction, employment, and autonomy.\n\t* Subject-level issues: machine ethics and artificial moral agency.\n4. The field of AI ethics is concerned with understanding the ethical implications of AI systems and developing frameworks for addressing these issues.\n5. Some of the key challenges and debates in AI ethics include:\n\t* The potential for AI systems to make decisions that have moral significance, such as taking lives or causing ha